In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

spark

In [3]:
rdd = spark.sparkContext.parallelize([1,2,3,4,56])

rdd.count()

5

In [4]:
dataList = [
    ("Java", 20000),
    ("Python", 100000),
    ("Scala", 3000)
]

rdd = spark.sparkContext.parallelize(dataList)

rdd

ParallelCollectionRDD[2] at readRDDFromFile at PythonRDD.scala:274

In [5]:
rdd2 = spark.sparkContext.textFile('qs_dataframe.ipynb')

In [7]:
rdd2

qs_dataframe.ipynb MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:0

In [8]:
data = [
    ('James','','Smith','1991-04-01','M',3000),
    ('Michael','Rose','','2000-05-19','M',4000),
    ('Robert','','Williams','1978-09-05','M',4000),
    ('Maria','Anne','Jones','1967-12-01','F',4000),
    ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]

df = spark.createDataFrame(data=data, schema=columns)

df.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [9]:
df.createOrReplaceTempView("PERSON_DATA")

df2 = spark.sql("SELECT * FROM PERSON_DATA")

df2

DataFrame[firstname: string, middlename: string, lastname: string, dob: string, gender: string, salary: bigint]

In [10]:
groupDF = spark.sql("SELECT gender, count(*) FROM PERSON_DATA GROUP BY gender")

groupDF.show()

+------+--------+
|gender|count(1)|
+------+--------+
|     M|       3|
|     F|       2|
+------+--------+



In [11]:
df = spark.readStream \
    .format('socket') \
    .option('host', 'localhost') \
    .option('port', '9090') \
    .load()

df.printSchema()

root
 |-- value: string (nullable = true)



22/01/01 09:25:02 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [13]:
from pyspark.sql.functions import explode, split

wordsDF = df.select(explode(split(df.value, ' ')).alias('word'))
count = wordsDF.groupBy('word').count()

query = count.writeStream \
    .format("console") \
    .outputMode("complete") \
    .start()

query.awaitTermination(3*60)

22/01/01 09:26:51 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0k/5497n5x55855vn0zx9hb__rc0000gq/T/temporary-7420a12d-762c-434b-9ccd-de8c139ea5ef. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/01/01 09:26:51 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|Hello|    1|
|world|    1|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|  you|    1|
|Hello|    1|
|  How|    1|
|  are|    1|
|world|    1|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|  you|    2|
| fine|    1|
|Hello|    1|
|  How|    1|
|thank|    1|
|  are|    1|
|world|    1|
|    I|    1|
|   am|    1|
+-----+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|  you|    3|
| fine|    1|
|  Are|    1|
|Hello|    1|
|  How|    1|
|thank|    1|
|  are|    1|
|world|    1|
|    I|    1|
|   am|    1|
|   OK|    1|
+-----+-----+



False

In [14]:
query